In [49]:
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 10 17:34:18 2020

@author: Guillermo Sánchez Gutiérrez-Cabello
"""
#%%
import numpy as np
import pandas as pd
import os
import sklearn
#pd.options.mode.chained_assignment = None  # default='warn'


from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()

import nltk
nltk.download("popular") # required to download the stopwords lists
from nltk.corpus import stopwords

spanish_stopwords = stopwords.words('spanish')
english_stopwords = stopwords.words('english')
#%%
mainpath = "C:/Users/guill/Documents/Universidad/PlataformaRefugiados/NAUTIA/DesarrolloPy/DataSetOriginales"

def fixBibliography(df):
    df = dfFix(df,"GENERAL INFORMATION - COUNTRY LEVEL")
    df.columns = ['GeneralInfo', 'CommunityCountry', 'RefugeeCountry']
    df.set_index('GeneralInfo', inplace = True)
    df = df.transpose()
    df = df.reset_index()
    return df

def dfFix(df,col1 = False,col2 = False):
    result = df.copy()
    if(col1):
        x = result.columns.get_loc(col1)
        result.drop(result.columns[0:x],axis = 1, inplace = True)
    if(col2):
        y = result.columns.get_loc(col2)
        result.drop(result.columns[y:],axis = 1, inplace = True)
    return result

def validColumn(cad):
    result = False
    if(cad == "index"):#Population
        result = True
    else:
        if(cad == "Type_of_settlement"):
            result = True
        else:
            if(cad == "General:settlement"):
                result = True
            else:
                if(cad == "general_info:_1_1_Choose_the_settlement"):
                    result = True
                else:
                    if(cad == "General_Information:Type_of_setlement"):
                        result = True
                    else:
                        if(cad == "General:Settlement"):
                            result = True
                        else:
                            if(cad == "Type_of_setlement"):
                                result = True
    return result

def dropRow(df,i):
    return df.drop(index = i)

def get_communityRows(df,cad,communityType): #la función pd.loc[] tiene un bug indiscriminado (https://github.com/pandas-dev/pandas/issues/8555)
    result = df
    if(communityType == 0):
        comm = "host_community"
    else:
        comm = "refugee_camp"
    for index, row in df.iterrows():
        if(row[cad] != comm):
            result = dropRow(result,index)
    return result

def setDataByIndex(df,communityType):
    array = df.columns
    i = 0
    df = df.replace("refugee","refugee_camp")
    df = df.replace("host_comunity","host_community")
    df = df.replace("RefugeeCountry","refugee_camp")
    df = df.replace("CommunityCountry","host_community")
    while(validColumn(array[i]) == False):
        i += 1
    df = get_communityRows(df,array[i],communityType)
    return df

def set_AllCSVtoDF(communityType):
    Bibliography = pd.read_excel(getPath(mainpath,"Bibliography_120220.xlsx"))
    Bibliography = fixBibliography(Bibliography)
    Bibliography = setDataByIndex(Bibliography,communityType)
    Entities = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Entities_Interview_results.csv")),communityType)
    LocalLeaders = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Local_leaders_v3_results.csv")),communityType)
    HouseHold = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Survey_household_v6_results.csv")),communityType)
    WomenGroup = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Women_Focus_Group2_results.csv")),communityType)
    SanitationInfra = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_V1_0_Sanitation_Infrastructre_results.csv")),communityType)
    Priorities = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Priorities_v3_results.csv")),communityType)
    GeneralForm = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_General_form_v3_results.csv")),communityType)
    PublicSpace = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Public_Space_results.csv")),communityType)
    WaterInf = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Water_Infrastructure_results.csv")),communityType)
    SanitationInf = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_V1_0_Sanitation_Infrastructre_results.csv")),communityType)
    WasteManagementInf = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Waste_Management_Infrastructure_results.csv")),communityType)
    EnergyINF = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Energy_Infrastructure_results.csv")),communityType)
    Business = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA1_0_Business_surveys_v3_results.csv")),communityType)
    MobilityINF = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0__Transport_servicesaccess_points_results.csv")),communityType) 
    ComunalServices = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Communal_Services_results.csv")),communityType) 
    GeneralCitizen = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_General_Citizen_Focus_Group_results.csv")),communityType)
    Shelter = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Shelter_results.csv")),communityType)
    FarmyardCrop = setDataByIndex(pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Farmyard_and_Crops_results.csv")),communityType)
    return Bibliography,Entities,LocalLeaders,HouseHold,WomenGroup,SanitationInfra,Priorities,GeneralForm,PublicSpace,WaterInf,EnergyINF,SanitationInf,WasteManagementInf,EnergyINF,Business,MobilityINF,ComunalServices,GeneralCitizen,Shelter,FarmyardCrop

def concatDF(df1,df2):
    return  pd.concat([df1,df2],axis = 1, ignore_index = True, sort = True)

def mkCSV(df,fileName):
    df = df.dropna(how = 'all')
    df *= 1   
    fileName = fileName.lower()
    df.to_csv('DataSetFinales/'+fileName,sep=',',header = False, index=False, encoding='utf-8')
    
def getPath(mainpath,filename):
    return os.path.join(mainpath, filename)

def getSubColumnNames(df,x):
    columns = df.columns
    array = []
    for column in columns:
        column = column[x:]
        array.append(column)
    return pd.DataFrame(array) 

def addInstitutionAndType(df,array1,array2,instType):
    df = df.dropna(axis = 1)
    df = np.array(df)
    for row in df:
        for elem in row:
            array1 = np.append(array1,elem)
            array2 = np.append(array2,instType)
    return array1,array2

def politicalActor(df1,df2,df3,df4,df5):
    institution = []
    instType = []

    institution, instType  = addInstitutionAndType(df1,institution,instType,'Public Institution')
    institution, instType  = addInstitutionAndType(df2,institution,instType,'Private Institution')
    institution, instType  = addInstitutionAndType(df3,institution,instType,'NGO')
    institution, instType  = addInstitutionAndType(df4,institution,instType,'International Agency')
    institution, instType  = addInstitutionAndType(df5,institution,instType,'Local')

    institution = pd.DataFrame(institution)
    institution = institution.reset_index(drop = True)
    instType = pd.DataFrame(instType)
    instType = instType.reset_index(drop = True)
    
    return concatDF(institution,instType)


def get_claveValor(df1,df2):
    array1 = np.array(df2)
    array2 = np.array(df1)
    result1 = []
    result2 = []
    i = 0
    for row in array1:
        var = array2[i]
        for elem in row:
            result1 = np.append(result1,elem)
            result2 = np.append(result2,var)
        i+=1
    result2 = pd.DataFrame(result2)
    result2 = result2.reset_index(drop = True)
    result1 = pd.DataFrame(result1)
    result1 = result1.reset_index(drop = True)
    return concatDF(result2,result1)

def get_number(df):
    df = np.array(df)
    array = np.array([])
    for column in df:
        for elem in column:
            array = np.append(array,elem)
    return (pd.DataFrame(array)).fillna(0) 

def get_valueBySector(df1,df2):
    df2 = df2.reset_index()
    array1 = np.array(df1)
    i = 0
    for row in array1:
        for elem in row:
            if(elem == False):
                df2 = dropRow(df2,i)
        i += 1
    df2 = df2.set_index('index')
    return df2


def separateValues(df):
    if(df.isnull().values.all(axis=0)):
        result = df
    else:
        array = np.array(df)
        count_vectorizer = CountVectorizer(stop_words = spanish_stopwords+english_stopwords)
        corpus = []
        for row in array:
            for elem in row:
                corpus = np.append(corpus,[elem])
        X = count_vectorizer.fit_transform(corpus)
        array = count_vectorizer.get_feature_names()
        result = pd.DataFrame(array)
    return result

def vectorizeValue(df):
    df = separateValues(df)
    year = np.array(['january','february','march','april','may','june','july','august','september','october','november','december'])
    result = np.array([],dtype = bool)
    df = np.array(df)
    for elem in year:
        flag = False
        for column in df:
            for month in column:
                if(column == elem):
                    flag = True
        if(flag):
            result = np.append(result,True)
        else:
            result = np.append(result,False)
    return pd.DataFrame(result)

def set_sector(df,sect, concat = True):
    sector = np.array([])
    df = df.dropna(how = 'all')
    df = np.array(df)
    for column in df:
        sector = np.append(sector,sect)
    sector = pd.DataFrame(sector)
    df = pd.DataFrame(df)
    if(concat):
        result = concatDF(sector,df)
    else:
        result = sector
    return result 

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]   

In [2]:
Bibliography,Entities,LocalLeaders,HouseHold,WomenGroup,SanitationInfra,Priorities,GeneralForm,PublicSpace,WaterInf,EnergyINF,SanitationInf,WasteManagementInf,EnergyINF,Business,MobilityINF,ComunalServices,GeneralCitizen,Shelter,FarmyardCrop = set_AllCSVtoDF(1)

In [3]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [4]:
Bibliography.shape

(1, 192)

In [5]:
Entities.shape

(3, 72)

In [6]:
LocalLeaders.shape

(3, 22)

In [7]:
HouseHold.shape

(24, 42)

In [8]:
WomenGroup.shape

(1, 27)

In [9]:
SanitationInfra.shape

(37, 14)

In [10]:
Priorities.shape

(2, 88)

In [11]:
GeneralForm.shape

(1, 19)

In [12]:
PublicSpace.shape

(2, 14)

In [13]:
WaterInf.shape

(30, 13)

In [14]:
SanitationInf.shape

(37, 14)

In [15]:
WasteManagementInf.shape

(1, 10)

In [16]:
EnergyINF.shape

(4, 38)

In [17]:
Business.shape

(16, 25)

In [18]:
MobilityINF.shape

(3, 10)

In [19]:
ComunalServices.shape

(43, 63)

In [20]:
GeneralCitizen.shape

(1, 43)

In [21]:
Shelter.shape

(23, 27)

In [22]:
FarmyardCrop.shape

(16, 18)

In [23]:
PublicSpace.shape

(2, 14)

In [169]:
def get_applianceDF(df):
    df1 = dfFix(df,1,2)
    df2 = dfFix(df,2)
    array = np.array([])
    appliance = np.array([])
    for row in np.array(df1):
        corpus = np.array([])
        for elem in row:
            corpus = np.append(corpus,[elem])
        count_vectorizer = CountVectorizer(stop_words = spanish_stopwords+english_stopwords, vocabulary = ['light_bulbs','mobile_phone','radio_music_pl','tv_dvd','laptop_tablet_','fridge','electrical_sto','others'])
        X = count_vectorizer.fit_transform(corpus)
        array = count_vectorizer.get_feature_names()
        for elem in array:
            appliance = np.append(appliance,elem)
    appliance = pd.DataFrame(appliance)
    hours = np.array([])
    for row in np.array(df2):
        for elem in row:
            hours = np.append(hours,elem)
    hours = pd.DataFrame(hours)
    hours = (hours.dropna())
    return concatDF(appliance,hours)
    

In [182]:
comercial = dfFix(Business,"Energy:electrical_appliances","Energy:money_electricity")
comercial = set_sector(comercial,"comercial")
comercial = get_applianceDF(comercial)
comercial = set_sector(comercial,"comercial")
residencial = dfFix(HouseHold,"Energy:Appliances","Energy:Elec_expen")
residencial = set_sector(residencial,"residencial")
residencial = get_applianceDF(residencial)
residencial = set_sector(residencial,"residencial")
comunitario = dfFix(ComunalServices,"Energy_Details:Electrical_Appliances:Devices","Construction_Details:Appropiate_Roof")
comunitario = set_sector(comunitario,"comunitario")
comunitario = get_applianceDF(comunitario)
comunitario = set_sector(comunitario,"comunitario")
INF_Appliance_has_Community = concatDF(comercial.T,concatDF(residencial.T,comunitario.T)).T
INF_Appliance_has_Community = INF_Appliance_has_Community[INF_Appliance_has_Community[1].notna()]
mkCSV(INF_Appliance_has_Community,"INF_Appliance_has_Community.csv")

In [183]:
INF_Appliance_has_Community

,0,1,2
0,comercial,light_bulbs,NaN
1,comercial,mobile_phone,1
2,comercial,radio_music_pl,NaN
3,comercial,tv_dvd,NaN
4,comercial,laptop_tablet_,NaN
5,comercial,fridge,NaN
6,comercial,electrical_sto,NaN
7,comercial,others,NaN
8,comercial,light_bulbs,9
9,comercial,mobile_phone,4


In [154]:
comunitario = dfFix(ComunalServices,"Energy_Details:Electrical_Appliances:Devices","Construction_Details:Appropiate_Roof")
comunitario = set_sector(comunitario,"comunitario")


In [155]:
comunitario

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,comunitario,lamp pc_or_tablet other,NaN,NaN,64,50,NaN,NaN,NaN,NaN,20,350,NaN,NaN,NaN,NaN,NaN,NaN,4,2000
1,comunitario,lamp,NaN,NaN,16,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,comunitario,lamp tv_or_dvd fridge,NaN,NaN,40,50,NaN,NaN,1,50,NaN,NaN,NaN,NaN,NaN,NaN,1,885,NaN,NaN
3,comunitario,lamp fridge other,NaN,NaN,15,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,300,2,1240
4,comunitario,lamp mobile,NaN,NaN,15,50,NaN,NaN,NaN,NaN,NaN,NaN,1,250,NaN,NaN,NaN,NaN,NaN,NaN
5,comunitario,other lamp mobile,NaN,NaN,50,50,NaN,NaN,NaN,NaN,NaN,NaN,8,250,NaN,NaN,NaN,NaN,19,NaN
6,comunitario,lamp fridge,NaN,NaN,20,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1411,NaN,NaN
7,comunitario,lamp pc_or_tablet other,NaN,NaN,110,50,NaN,NaN,NaN,NaN,11,300,NaN,NaN,NaN,NaN,NaN,NaN,2,900
8,comunitario,lamp fridge other,NaN,NaN,2,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,11500,4,2100
9,comunitario,lamp other,NaN,NaN,28,150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2100


In [139]:
df1 = dfFix(residencial,1,2)
df2 = dfFix(residencial,2)